In [ ]:
import sys, os, pathlib
sys.path.append("/root/shared/gitrepos/smart-comp-sci/utils")
sys.path.append("/root/shared/gitrepos/smart-comp-sci/mechanotransduction-example")
import smart_analysis
from mechanotransduction_ode import mechanotransduction_ode_calc
from matplotlib import pyplot as plt
plt.style.use(str(pathlib.Path.cwd() / ".." / "utils" / "smart_plots.mplstyle"))
import numpy as np

Analyze mechanotransduction simulations (provide path to folder containing all cases tested).

In [ ]:
files_dir = "/root/shared/gitrepos/smart-comp-sci/analysis_data/mechanotransduction/mech_new_results"
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "mechanotransduction" / "mech_npy_files"
mesh_folder = "/root/shared/write-meshes/mechanotransduction/meshes"
npy_dir.mkdir(exist_ok=True)
test_folders = os.listdir(files_dir)
shapes = ["circle", "star", "rect"]
full3dstr = ["full3dtrue", "full3dfalse"]
for i in range(len(test_folders)):
    shape_logic = [shape in test_folders[i] for shape in shapes]
    shape_idx = np.nonzero(shape_logic)[0]
    full3d_logic = [full3d in test_folders[i] for full3d in full3dstr]
    full3d_idx = np.nonzero(full3d_logic)[0]
    results_folder = f"{files_dir}/{test_folders[i]}"
    mesh_file = ""
    if len(shape_idx)==1 and len(full3d_idx)==1:
        for mesh_folder_test in os.listdir(mesh_folder):
            if (shapes[shape_idx[0]] in mesh_folder_test and
                full3dstr[full3d_idx[0]] in mesh_folder_test):
                mesh_file = f"{mesh_folder}/{mesh_folder_test}/spreadCell_mesh.h5"
                break
    if mesh_file == "":
        print(f"Could not find mesh file for {test_folders[i]}")
        continue

    if "circle" in results_folder and "full3dfalse" in results_folder:
        axisymm = True
    else:
        axisymm = False
    tVec, results_all = smart_analysis.analyze_all(
        mesh_file=mesh_file, results_path=results_folder, display=False, axisymm=axisymm)
    results_all.insert(0, tVec) # add time as first element in list
    max_length = len(tVec)
    for j in range(len(results_all)):
        if len(results_all[j]) > max_length:
            max_length = len(results_all[j])
    for j in range(len(results_all)):
        num_zeros = max_length - len(results_all[j])
        for k in range(num_zeros):
            results_all[j].append(0)
    np.save(npy_dir / f"{test_folders[i]}_redo_results.npy", results_all)

Plot average F-actin for each of the 3 different cell geometries.

In [ ]:
contact_rad = 13
stiffness_vec = [70000000]
shape_vec = ["circle", "rect", "star"]
symm_vec = [0, 0.25, 0.5]
cyto_vol = 480
pm_area = 320
nuc_vol = 70.6
nm_area = 58.5
Ac = 133
npy_dir = pathlib.Path.cwd() / ".." / "analysis_data" / "mechanotransduction" / "mech_npy_files"
# geoParam = [cyto_vol, nuc_vol, pm_area, nm_area, Ac]
geoParam = [[1925.03/4, nuc_vol, 1294.5/4, nm_area, Ac], [480.1, nuc_vol, 332.7, nm_area, Ac], [952.9/2, nuc_vol, 681.1/2, nm_area, Ac]]
var_names_all = ["Cofilin_P", "Fak", "mDia", "LaminA", "FActin", "RhoA_GTP", "mDia_A", "NPC_A", "GActin", "NPC",
        "ROCK_A", "Myo", "Cofilin_NP", "LaminA_p", "YAPTAZ_nuc", "pFak", "YAPTAZ_phos", "YAPTAZ", "RhoA_GDP", "LIMK",
        "Myo_A", "ROCK", "Positionboolean", "LIMK_A", "MRTF", "MRTF_nuc"]
var_names_saved = ["Cofilin_NP", "Emod", "FActin", "GActin", "LaminA", "LIMK_A", "mDia_A", 
                   "Myo_A", "NPC_A", "pFAK", "RhoA_GDP", "RhoA_GTP", "ROCK_A", "YAPTAZ", "YAPTAZ_nuc", "YAPTAZ_phos"]
plot_name = "FActin"
ode_idx = var_names_all.index(plot_name)
results_idx = var_names_saved.index(plot_name) + 1 # add one because time is first el
fig, ax = plt.subplots(figsize=(5,3))
full3d = False
for stiffness in stiffness_vec:
    for i in range(len(shape_vec)):
        file_cur = f"{str(npy_dir)}/{shape_vec[i]}_E{stiffness}_full3d{full3d}_results.npy"
        results_cur = np.load(file_cur)
        plt.plot(results_cur[0]/60, results_cur[results_idx],label=f"{shape_vec[i]} {stiffness} kPa")

plt.ylabel("F-actin (μM)")
plt.xlabel('Time (min)')
plt.xlim([0, 60])

plt.savefig(npy_dir / "Factin.pdf", format="pdf")

Plot average YAP/TAZ nuclear to cytosolic ratio (N/C) for each of the 3 different cell geometries.

In [ ]:
stiffness_vec = [70000000]#[0.1, 5.7, 70000000.0]
shape_vec = ["circle", "rect", "star"]
symm_vec = [0, 0.25, 0.5]
# plot YAP/TAZ N/C ratio
plot_names = ["YAPTAZ_phos", "YAPTAZ", "YAPTAZ_nuc"]
ode_idx = []
results_idx = []
for name in plot_names:
    results_idx.append(var_names_saved.index(name) + 1) # add one because time is first el
    ode_idx.append(var_names_all.index(name))
fig, ax = plt.subplots(figsize=(5,3))
full3d = False
for stiffness in stiffness_vec:
    for i in range(len(shape_vec)):
        file_cur = f"{str(npy_dir)}/{shape_vec[i]}_E{stiffness}_full3d{full3d}_results.npy"
        results_cur = np.load(file_cur)
        yaptaz_curratio = results_cur[results_idx[2]]/(results_cur[results_idx[0]]+results_cur[results_idx[1]])
        plt.plot(results_cur[0]/60, yaptaz_curratio, label=f"{shape_vec[i]} on glass")#{stiffness} kPa")

plt.ylabel("YAP/TAZ N/C")
plt.xlabel('Time (min)')
plt.xlim([0, 60])

plt.savefig(f"{str(npy_dir)}/YAPTAZ.pdf", format="pdf")

Generate postprocessing plots comparing well-mixed SMART simulation to ODE solution.

In [ ]:
import postprocess, json
output_folder = pathlib.Path.cwd() / ".." / "analysis_data" / "mechanotransduction" / "mech-refinement"
results_file = output_folder / "results_mechanotransduction.json"
print(f"Load results from {results_file}")
all_results = [postprocess.Data(**r) for r in json.loads(results_file.read_text())]
postprocess.plot_data(all_results, output_folder, format=".pdf")